<a href="https://colab.research.google.com/github/lblogan14/colab/blob/master/tensorboard_on_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Use TensorBoard in Colab
This is an example of how to set up TensorBoard on Colab.

##Install ngrok

In [1]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2018-11-05 20:25:18--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.0.94.50, 52.201.75.180, 52.55.191.55, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.0.94.50|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5363700 (5.1M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]   5.11M  19.6MB/s    in 0.3s    

2018-11-05 20:25:19 (19.6 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [5363700/5363700]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


##Run Tensorboard
assuming the TensorBoard log path is "./log"

In [0]:
LOG_DIR = './log'
get_ipython().system_raw(
      'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'.format(LOG_DIR))

##Run ngrok
run ngrok to tunnel TensorBoard port 6006 to the outside world

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

## Get URL
access the colab TensorBoard web page


In [4]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://a9c10463.ngrok.io


#Set up a simple Keras network 

In [0]:
'''Trains a simple convnet on the MNIST dataset.

Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.callbacks import TensorBoard

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])


tbCallBack = TensorBoard(log_dir='./log', histogram_freq=1,
                         write_graph=True,
                         write_grads=True,
                         batch_size=batch_size,
                         write_images=True)

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[tbCallBack])
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Using TensorFlow backend.


11493376/11490434 [==============================] - 0s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 13s 220us/step - loss: 0.2625 - acc: 0.9198 - val_loss: 0.0523 - val_acc: 0.9840
Epoch 2/12
60000/60000 [==============================] - 10s 169us/step - loss: 0.0906 - acc: 0.9736 - val_loss: 0.0387 - val_acc: 0.9863
Epoch 3/12
60000/60000 [==============================] - 10s 168us/step - loss: 0.0661 - acc: 0.9802 - val_loss: 0.0353 - val_acc: 0.9877
Epoch 4/12
60000/60000 [==============================] - 10s 164us/step - loss: 0.0559 - acc: 0.9826 - val_loss: 0.0303 - val_acc: 0.9892
Epoch 5/12
60000/60000 [==============================] - 10s 164us/step - loss: 0.0476 - acc: 0.9857 - val_loss: 0.0271 - val_acc: 0.9908
Epoch 6/12
60000/60000 [==============================] - 10s 171us/step - loss: 0.0420 - acc: 0.9872 - val_loss: 0.0

While this model is training, you can open the previous ngrok link, it should show the TensorBoard page.

##Optionally check tensorboard and ngrok PIDs and stop them

In [0]:
ps aux | grep tensorboard

In [0]:
!ps aux | grep ngrok

In [0]:
!kill 155

##Optionally clear the tensorboard records directory

In [0]:
!rm -rf log